In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

# 1. 경로 설정
DATA_PATH = "../../data/raw/"
OOF_PATH = "./oof_data/"
SUB_PATH = "./submissions/"

# 2. 정답 및 인덱스 정렬
train_df = pd.read_csv(f"{DATA_PATH}train.csv")
m7_mask = (train_df.familysize <= 50)
a_cols = [col for col in train_df.columns if col.endswith('A') and col.startswith('Q')]
m7_mask &= (train_df[a_cols].std(axis=1) > 0)
m7_indices = train_df[m7_mask].index
train_y = (2 - train_df.loc[m7_indices, 'voted'].to_numpy()).astype(np.float32)

# 3. OOF 로드 (정방향: Large = Voted)
m1_oof_raw = np.load(f"{OOF_PATH}exp11_sota_AUC_0.77212.npy")
m1_indices = train_df[train_df.familysize <= 50].index
m1_oof = pd.DataFrame(m1_oof_raw, index=m1_indices, columns=['val']).reindex(m7_indices)['val'].values
m7_oof = np.load(f"{OOF_PATH}exp25_m7_Refined_AUC_0.77375.npy")

# 4. Rank 변환
m1_rank = pd.Series(m1_oof).rank(pct=True).values
m7_rank = pd.Series(m7_oof).rank(pct=True).values

# 5. [Fine-tuning] 4:6 가중치 앙상블
# 로컬 성능이 더 좋은 m7에 60% 가중치를 부여합니다.
ensemble_rank = (m1_rank * 0.4) + (m7_rank * 0.6)
final_auc = roc_auc_score(train_y, ensemble_rank)

print("-" * 50)
print(f"Exp 27: Rank Ensemble (m1:0.4, m7:0.6)")
print(f"Final Local AUC: {final_auc:.5f}")
print("-" * 50)

# 6. 테스트 데이터 적용 (Sub LB 마지막 제출용)
m1_test = pd.read_csv(f"{SUB_PATH}0130-1923.csv")['voted'] 
m7_test = pd.read_csv(f"{SUB_PATH}m7_Refined_test_preds.csv")['voted']

# m1 방향 복구 (Small -> Large) 및 랭킹
m1_t_rank = (2.0 - m1_test).rank(pct=True).values
m7_t_rank = m7_test.rank(pct=True).values

# 가중치 적용
final_test_rank = (m1_t_rank * 0.4) + (m7_t_rank * 0.6)

# 최종 제출: 리더보드 기준(Small = Voted)으로 반전
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
submission['voted'] = 2.0 - final_test_rank 

output_name = f"{SUB_PATH}27_m1_m7_Rank_4to6_{final_auc:.5f}.csv"
submission.to_csv(output_name, index=False)
np.save(f"{OOF_PATH}exp27_m1_m7_Rank_4to6_AUC_{final_auc:.5f}.npy", ensemble_rank)

print(f"Submission Created: {output_name}")

--------------------------------------------------
Exp 27: Rank Ensemble (m1:0.4, m7:0.6)
Final Local AUC: 0.77379
--------------------------------------------------
Submission Created: ./submissions/27_m1_m7_Rank_4to6_0.77379.csv
